In [25]:
import sys
sys.path.insert(0, '../')

In [26]:
import torchtext.data as data
import numpy as np
import training_utils as tu
from torch.utils.tensorboard import SummaryWriter
from models import CNN
from importlib import reload
from pathlib import Path

In [72]:
process_text = {
    
}

process_labels = {
    
}

TEXT = data.Field(**process_text)
LABEL = data.LabelField(**process_labels)
fields = {'category_id': ('label', LABEL), 'lemmatized': ('text', TEXT)}

In [46]:
MAX_VOCAB_SIZE = 25_000

BATCH_SIZE = 64


EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
DROPOUT = 0.5



In [73]:
for name, field in fields.values():
    print(field)

In [68]:
fields = (TEXT, LABEL, 1, 2)

In [71]:
(t, l), others = fields

ValueError: too many values to unpack (expected 2)

In [53]:
import torch

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(1)

In [48]:
import models
reload(models)
from models import CNN

In [32]:
train_data, valid_data = data.TabularDataset.splits(
                                        path = '../data/json',
                                        train = 'train.json',
                                        validation = 'valid.json',
                                        format = 'json',
                                        fields = fields
)

test_data = data.TabularDataset(
                                path = '../data/json/test.json',
                                format = 'json',
                                fields = {'lemmatized': ('text', TEXT)}
)

In [33]:
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)
print(f'Vocab size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocab size: 25002
Number of classes: 54


In [34]:
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda ex: len(ex.text),
    sort_within_batch = False,
    device = device)

In [49]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(LABEL.vocab)
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, OUTPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, DROPOUT, PAD_IDX)

In [54]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [51]:
tu.count_parameters(model)

2606754

In [60]:
writer=SummaryWriter('../runs/cnn')

In [61]:
reload(tu)
tu.train_model(model, train_iterator, valid_iterator,
            optimizer, criterion, 'cnn', n_epochs=25, writer=writer)

0.5746569367247372 0.8680419390108071 0.37117139667499044 0.8915103185720392
Epoch: 1 | Epoch Time: 3m 10:s 
0.5956889365008405 0.8669628267973856 0.3586113110451327 0.8947481842363635
Epoch: 2 | Epoch Time: 3m 3:s 
0.5990369395310291 0.867071759272245 0.35321969801688197 0.8962499062343641
Epoch: 3 | Epoch Time: 3m 5:s 
0.6016504764934284 0.8663398692810458 0.3463630177498273 0.8973064363480082
Epoch: 4 | Epoch Time: 3m 0:s 
0.6155405799815663 0.8664453975905001 0.3403630869057925 0.8993371141114681
Epoch: 5 | Epoch Time: 3m 4:s 
0.6293050361444164 0.8671840958735522 0.33202765581629834 0.9015282869903962
Epoch: 6 | Epoch Time: 3m 0:s 
0.6450433987844126 0.8670547385620915 0.3277121018743151 0.9029804874058739
Epoch: 7 | Epoch Time: 3m 3:s 
0.6509105457565169 0.86683687365133 0.3212897140452978 0.904359765936491
Epoch: 8 | Epoch Time: 3m 0:s 
0.6668744451968985 0.8677866285532908 0.3183537279262537 0.9052203027889179
Epoch: 9 | Epoch Time: 3m 3:s 
0.6863518652500086 0.8643995098039216